In [1]:
"""
4000个学生在某个时间段内预选1335门课（每个学生允许预选0-1335门，预选哪门课随机），预选阶段结束后，每个学生关于每门课得到一个随机数：
若某门课被预选，则随机数ran = randint(1,a+b*预选课门数)；若未被预选则ran = Inf。然后针对每门课，对全部400名学生的ran值按从小到大排序，前40名学生
可选到该门课（不足40也行），进而得到每个学生的实选课程。选课结束后，统计每个学生的选课情况，调整参数a,b（预计a=10,b=3)，使得预选课程门数为6-10
的学生选到1到8门课的概率最大。

1. 随机取样函数 np.random.randint(1,a+b*class_num[i]) 保证不重复
2. 每门课都要随机取ran值
3. 考虑选上1-8门课的概率
"""

'\n4000个学生在某个时间段内预选1335门课（每个学生允许预选0-1335门【假定不超过20】，预选哪门课随机），预选阶段结束后，每个学生关于每门课得到一个随机数：\n若某门课被预选，则随机数ran = randint(1,a+b*预选课门数)；反正则ran = Inf。然后针对每门课，对全部400名学生的ran值按从小到大排序，前40名学生\n可选到该门课（不足40也行），进而得到每个学生的实选课程。选课结束后，统计每个学生的选课情况，调整参数a,b（预计a=10,b=3)，使得预选课程门数为6-8或\n6-10的学生至少选到一门课的概率最大。\n\n1. 随机取样函数 np.random.randint(1,a+b*class_num[i]) 保证不重复\n2. 每门课都要随机取ran值\n3. 考虑选上1-8门课的概率\n'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"] # 解决中文标题，坐标轴名，刻度名等问题
plt.rcParams["axes.unicode_minus"]=False

total_student=4000 # 总学生数
total_class=1335 # 总课程数
student_limit = 40

a=12
for b in range(1,5):
        class_num=np.array([]) # 每个学生预选课程数
        for j in range(4000):
            class_num = np.append(class_num, np.random.randint(1,1336))

        class_={}
        for j in range(4000):
            class_[j]=np.zeros(1335)
            r=np.random.choice(list(range(1335)), size=int(class_num[j]), replace=False)
            length = len(r)
            for k in range(length):
                class_[j][r[k]]=1

        class_0 = pd.DataFrame(class_) # 预选课程表
        class_1 = pd.DataFrame(np.zeros([1335,4000]),columns=class_0.columns) # 实选课程表
        for i in range(class_0.shape[0]): # 行1335
            ran_ = np.array([]) # 每门课下每个学生随机数
            for j in range(class_0.shape[1]): # 列4000
                if class_0[j][i]==0:
                    ran_ = np.append(ran_,float('inf')) # 没选该门课则随机数无穷大
                else:
                    ran_ = np.append(ran_, np.random.randint(1,a+b*class_num[j]))
            ran_sort = np.sort(ran_) # 返回从小到大随机数排序
            ran_argsort = np.argsort(ran_) # 返回排序后对应索引
            flag = 40 
            for k in range(40):
                if ran_sort[k]==float('Inf'):
                    flag = k
                    break
            class_1.loc[i,ran_argsort[:flag]]=1

        prob = {} # 预选对应t-t+4门课的学生，实选到num门课的概率
        for num in range(1,9):
            for t in range(1,1336,5):
                count0 = len(class_num[(class_num>=t)&(class_num<=t+4)]) # 预选t-t+4门课的学生数
                index = np.where((class_num>=t)&(class_num<=t+4))[0] # 预选t-t+4门课的学生索引
                count1 = 0 # 上述学生选到num门课的数目
                for idx in index:
                    if class_1[idx].sum()==num:
                        count1 = count1+1
                if count0>0:
                    prob[t] = count1/count0
                else:
                    prob[t] = 0

            # 可视化
            prob_ = []
            for k in prob:
                prob_.append(prob[k])
            
            # 求实选中num门课概率最大时，应该预选多少门课
            prob_max = max(prob_)
            idx_max = prob_.index(max(prob_))*5+1
            print('当参数a='+str(a)+" b="+str(b)+"时，"+
                '预选'+str(idx_max)+'-'+str(idx_max+4)+'门课的学生,实选到'+
                str(num)+'门课的概率最大，为:'+str(prob_max))
        print('-------------------------------------------------')

    # plt.figure(figsize=[8,8])
    # plt.scatter(list(range(1,1336,5)), prob_)
    # plt.xlabel('预选课程数区间t-t+4')
    # plt.ylabel('选到'+str(num)+'门课概率')
    # plt.savefig('选到'+str(num)+'门课概率分布.png')


In [ ]:
prob = {} # 预选对应t-t+4门课的学生，至少实选到num门课的概率
for num in range(1,9):
    for t in range(1,1336,5):
        count0 = len(class_num[(class_num>=t)&(class_num<=t+4)]) # 预选t-t+4门课的学生数
        index = np.where((class_num>=t)&(class_num<=t+4))[0] # 预选t-t+4门课的学生索引
        count1 = 0 # 上述学生选到num门课的数目
        for idx in index:
            if class_1[idx].sum()>=num:
                count1 = count1+1
        if count0>0:
            prob[t] = count1/count0
        else:
            prob[t] = 0

    # 可视化
    prob_ = []
    for k in prob:
        prob_.append(prob[k])

    plt.figure(figsize=[8,8])
    plt.scatter(list(range(1,1336,5)), prob_)
    plt.xlabel('预选课程数区间t-t+4')
    plt.ylabel('至少选到'+str(num)+'门课概率')
    plt.savefig('至少选到'+str(num)+'门课概率分布.png')